# M2177.003100 Deep Learning <br> Assignment #1 Part 3: Playing with Neural Networks by PyTorch

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

Previously in `Assignment1-1_Data_Curation.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using PyTorch.

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **part 1 - 3**, run the *CollectSubmission.sh* script with your **Student number** as input argument. <br>
This will produce a compressed file called *[Your student number].tar.gz*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* &nbsp; 20\*\*-\*\*\*\*\*)

## Load datasets

First reload the data we generated in `Assignment1-1_Data_Curation.ipynb`.

In [4]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import torch
from six.moves import cPickle as pickle
from six.moves import range
import os

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#pickle_file = 'data/notMNIST.pickle'
pickle_file = '/content/drive/My Drive/20년 2학기/딥러닝/Assignment1/data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- unnormalize data
- data as a flat matrix

In [7]:
image_size = 28
num_labels = 10

def reformat(dataset):
    dataset = dataset * 255.0 + 255.0/2
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    return dataset

train_dataset = reformat(train_dataset)
valid_dataset = reformat(valid_dataset)
test_dataset = reformat(test_dataset)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000,)
Validation set (10000, 784) (10000,)
Test set (10000, 784) (10000,)


## PyTorch tutorial: Fully Connected Network

We're first going to train a **fully connected network** with *1 hidden layer* with *1024 units* using stochastic gradient descent (SGD).

In [8]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter

import torch.optim as optim

### first, define NotMNIST dataset class.  
- dataset class inherits torch.utils.data.Dataset class  
- every dataset class should define \_\_len\_\_() and \_\_getitem\_\_()

In [9]:
class NotMNIST(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data_idx = self.data[idx]
        label_idx = self.label[idx]
        return data_idx, label_idx


notmnist_train = NotMNIST(train_dataset, train_labels)
notmnist_valid = NotMNIST(valid_dataset, valid_labels)
notmnist_test = NotMNIST(test_dataset, test_labels)

print('training set length: ', len(notmnist_train))
print('validation set length: ', len(notmnist_valid))
print('test set length: ', len(notmnist_test))

training set length:  200000
validation set length:  10000
test set length:  10000


### Then, make dataloader using NotMNIST dataset objects  
Note that torch.utils.data.DataLoader is a subclass of Iterable, which means it can be used with 'for' statement (for more detailed explanation of Iterable, refer to https://shoark7.github.io/programming/python/iterable-iterator-generator-in-python)  

In [10]:
batch_size = 64

train_loader = DataLoader(dataset=notmnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
valid_loader = DataLoader(dataset=notmnist_valid, batch_size=len(notmnist_valid), shuffle=True)
test_loader = DataLoader(dataset=notmnist_test, batch_size=len(notmnist_test), shuffle=True)

from collections.abc import Iterable
print(issubclass(DataLoader, Iterable))

print('train loader length: ', len(train_loader)) # same as len(dataset) // batch_size
print('valid loader length: ', len(valid_loader))
print('test loader length: ', len(test_loader))

True
train loader length:  3125
valid loader length:  1
test loader length:  1


### Define Naive Linear model
- model should inherit nn.Module
- implement feed forward by overriding **forward** method of nn.Module

In [15]:
torch.manual_seed(1)

class NaiveLinear(nn.Module):
    
    def __init__(self, in_features, out_features):
        super(NaiveLinear, self).__init__()
        self.weight = Parameter(torch.Tensor(in_features, out_features))
        self.bias = Parameter(torch.Tensor(out_features))
        torch.nn.init.uniform_(self.weight, -1.0, 1.0)
        torch.nn.init.zeros_(self.bias)
    
    def forward(self, input):
        return torch.matmul(input, self.weight) + self.bias

In [16]:
class Model(nn.Module):
    
    def __init__(self, in_features, nn_hidden, num_labels):
        super(Model, self).__init__()
        self.fc1 = NaiveLinear(in_features, nn_hidden)
        self.fc2 = NaiveLinear(nn_hidden, num_labels)
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

In [17]:
nn_hidden = 1024

model = Model(image_size*image_size, nn_hidden, num_labels)

# move model to GPU
model.cuda()
# print model, initialized weight, grad buffer
print(model)
print(model.fc1.weight.data)
print(model.fc1.bias.grad)

Model(
  (fc1): NaiveLinear()
  (fc2): NaiveLinear()
)
tensor([[ 0.5153, -0.4414, -0.1939,  ..., -0.0334, -0.3184, -0.6335],
        [ 0.1658,  0.0407,  0.1526,  ..., -0.9122, -0.9774, -0.8611],
        [-0.5218,  0.6270,  0.8866,  ..., -0.1884,  0.3789, -0.5059],
        ...,
        [ 0.0949, -0.5369,  0.1770,  ...,  0.7149,  0.9331, -0.7348],
        [-0.7956,  0.9141,  0.3562,  ..., -0.1767, -0.8524,  0.0262],
        [-0.8096, -0.3422, -0.3679,  ...,  0.1363,  0.2865,  0.6339]],
       device='cuda:0')
None


Now, define loss function and optimizer

In [18]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(params=model.parameters(), lr=0.005)

Let's run this computation and iterate:

In [36]:
epochs = 10
log_step = 1000

def accuracy(logits, labels):
    logits = logits.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    
    return (100.0 * np.sum(np.equal(np.argmax(logits, 1), labels))
          / logits.shape[0])


# train_model
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for idx, data in enumerate(train_loader):
        images_flatten, labels = data[0].cuda(), data[1].long().cuda()
        logits = model(images_flatten)

        optimizer.zero_grad()
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        if (idx % log_step) == log_step-1:
            print(f'epoch: {epoch+1} [{idx + 1} / {len(train_loader)}]\t train_loss: {loss.item():.3f}\t train_accuracy: {accuracy(logits, labels):.1f}\n')


# evaluate model
def evaluate(model, test_loader):
    with torch.no_grad():
        for _, data in enumerate(test_loader):
            test_images_flatten, test_labels = data[0].cuda(), data[1].cuda()
            test_logits = model(test_images_flatten)

        print(f'accuracy: {accuracy(test_logits, test_labels):.1f}\n')
        #return accuracy(test_logits, test_labels)
    

In [21]:
for epoch in range(epochs):
    train(model, train_loader, optimizer, criterion, epoch)
    print('-------- validation --------')
    evaluate(model, valid_loader)

            
print('-------- test ---------')
evaluate(model, test_loader)

    
# save model
torch.save(model.state_dict(), '/content/drive/My Drive/20년 2학기/딥러닝/Assignment1/model_checkpoints/naive_model_final.pt')
print('naive model saved')

epoch: 1 [1000 / 3125]	 train_loss: 2.025	 train_accuracy: 67.2

epoch: 1 [2000 / 3125]	 train_loss: 1.840	 train_accuracy: 70.3

epoch: 1 [3000 / 3125]	 train_loss: 1.701	 train_accuracy: 79.7

-------- validation --------
accuracy: 74.8

epoch: 2 [1000 / 3125]	 train_loss: 2.291	 train_accuracy: 68.8

epoch: 2 [2000 / 3125]	 train_loss: 1.746	 train_accuracy: 81.2

epoch: 2 [3000 / 3125]	 train_loss: 2.265	 train_accuracy: 71.9

-------- validation --------
accuracy: 75.9

epoch: 3 [1000 / 3125]	 train_loss: 1.974	 train_accuracy: 67.2

epoch: 3 [2000 / 3125]	 train_loss: 1.044	 train_accuracy: 81.2

epoch: 3 [3000 / 3125]	 train_loss: 0.700	 train_accuracy: 79.7

-------- validation --------
accuracy: 75.5

epoch: 4 [1000 / 3125]	 train_loss: 1.962	 train_accuracy: 68.8

epoch: 4 [2000 / 3125]	 train_loss: 1.849	 train_accuracy: 76.6

epoch: 4 [3000 / 3125]	 train_loss: 1.297	 train_accuracy: 78.1

-------- validation --------
accuracy: 75.0

epoch: 5 [1000 / 3125]	 train_loss: 1.99

So far, you have built the model in a naive way. However, PyTorch provides a linear module named nn.Linear for your convenience. 

From now on, build the same model as above using layers module.

You can also build model using nn.Sequential()

In [22]:
model_layer = nn.Sequential(
            # neural network using nn.Linear
            nn.Linear(image_size * image_size, nn_hidden),
            nn.Tanh(),
            nn.Linear(nn_hidden, num_labels),
            nn.LogSoftmax(dim=1)
            )

model_layer.cuda()

Sequential(
  (0): Linear(in_features=784, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=10, bias=True)
  (3): LogSoftmax(dim=1)
)

In [23]:
criterion_layer = nn.NLLLoss()
optimizer_layer = optim.SGD(model_layer.parameters(), lr=0.005)

In [24]:
for epoch in range(epochs):
    train(model_layer, train_loader, optimizer_layer, criterion_layer, epoch)
    print('-------- validation --------')
    evaluate(model_layer, valid_loader)

            
print('-------- test ---------')
evaluate(model_layer, test_loader)

    
# save model
torch.save(model_layer.state_dict(), '/content/drive/My Drive/20년 2학기/딥러닝/Assignment1/model_checkpoints/layer_model_final.pt')
print('layer_model saved')

epoch: 1 [1000 / 3125]	 train_loss: 0.480	 train_accuracy: 87.5

epoch: 1 [2000 / 3125]	 train_loss: 0.550	 train_accuracy: 82.8

epoch: 1 [3000 / 3125]	 train_loss: 0.690	 train_accuracy: 78.1

-------- validation --------
accuracy: 82.2

epoch: 2 [1000 / 3125]	 train_loss: 0.558	 train_accuracy: 84.4

epoch: 2 [2000 / 3125]	 train_loss: 0.419	 train_accuracy: 89.1

epoch: 2 [3000 / 3125]	 train_loss: 0.782	 train_accuracy: 78.1

-------- validation --------
accuracy: 82.6

epoch: 3 [1000 / 3125]	 train_loss: 0.825	 train_accuracy: 75.0

epoch: 3 [2000 / 3125]	 train_loss: 0.454	 train_accuracy: 87.5

epoch: 3 [3000 / 3125]	 train_loss: 0.612	 train_accuracy: 79.7

-------- validation --------
accuracy: 82.0

epoch: 4 [1000 / 3125]	 train_loss: 0.582	 train_accuracy: 82.8

epoch: 4 [2000 / 3125]	 train_loss: 0.935	 train_accuracy: 76.6

epoch: 4 [3000 / 3125]	 train_loss: 0.612	 train_accuracy: 81.2

-------- validation --------
accuracy: 82.2

epoch: 5 [1000 / 3125]	 train_loss: 0.81

---
Problem 1
-------

**Describe below** why there is a difference in an accuracy between the model using nn.Linear and the model which is built in a naive way. **explain simply**  
You can refer to PyTorch documentation(https://pytorch.org/docs/stable/index.html) to check nn.Linear() implementation





---

The weight and bias values are initialized better, by referencing the input and output size, instead of just zeros or uniform distribution.

































>
/Assignment1_sol/
Name
Last Modified

---
Problem 2
-------

Try to get the best performance you can using a multi-layer model! (It doesn't matter whether you implement it in a naive way or using layer module. HOWEVER, you CANNOT use other type of layers such as conv.) 

The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.kr/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595). You may use techniques below.

1. Experiment with different hyperparameters: epochs, learning rate, etc.
2. We used a fixed learning rate epsilon for gradient descent. Implement an annealing schedule for the gradient descent learning rate ([more info](http://cs231n.github.io/neural-networks-3/#anneal)). *Hint*. Try using `torch.optim.lr_scheduler.ExponentialLR()`.    
3. We used a tanh activation function for our hidden layer. Experiment with other activation functions included in PyTorch.
4. Extend the network to multiple hidden layers. Experiment with the layer sizes. Adding another hidden layer means you will need to adjust the code. 
5. Introduce and tune regularization method (e.g. L2 regularization) for your model. Remeber that L2 amounts to adding a penalty on the norm of the weights to the loss. The right amount of regularization should imporve your validation / test accuracy.


**Evaluation:** You will get full credit if your best test accuracy exceeds <font color=red>$93\%$</font>. Save your best perfoming model using saver.  
**<font color=red>Save your model in directory ./model_checkpoints</font>** (Refer to the cell above)  
**<font color=red>Please follow format as problem2_(Student Number)</font>** (e.g. set path as './model_checkpoints/problem2_2020-23456')

---

In [37]:
epochs = 20
log_step = 1000
nn_hidden1 = 512
nn_hidden2 = 256
nn_hidden3 = 128
nn_hidden4 = 64
nn_hidden5 = 32

model_new = nn.Sequential(
            # neural network using nn.Linear
            nn.Linear(image_size * image_size, nn_hidden1),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Linear(nn_hidden1, nn_hidden2),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Linear(nn_hidden2, nn_hidden3),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Linear(nn_hidden3, nn_hidden4),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Linear(nn_hidden4, nn_hidden5),
            nn.LeakyReLU(),
            #nn.Dropout(p=0.1),
            nn.Linear(nn_hidden5, num_labels),
            nn.LogSoftmax(dim=1)
            )

model_new.cuda()

criterion_new = nn.NLLLoss()
optimizer_new = optim.SGD(model_new.parameters(), lr=0.01, momentum=0.5)
#optimizer_93 = optim.Adam(model_93.parameters(), lr=0.005)
#scheduler_93 = optim.lr_scheduler.ExponentialLR(optimizer_93, gamma= 0.99)

# train_model
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for idx, data in enumerate(train_loader):
        images_flatten, labels = data[0].cuda(), data[1].long().cuda()
        logits = model(images_flatten)

        #scheduler.step()

        optimizer.zero_grad()
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        if (idx % log_step) == log_step-1:
            print(f'epoch: {epoch+1} [{idx + 1} / {len(train_loader)}]\t train_loss: {loss.item():.3f}\t train_accuracy: {accuracy(logits, labels):.1f}\n')

for epoch in range(epochs):
    train(model_new, train_loader, optimizer_new, criterion_new, epoch)
    print('-------- validation --------')
    evaluate(model_new, valid_loader)

print('-------- test ---------')
evaluate(model_new, test_loader)


# save model
torch.save(model_93.state_dict(), '/content/drive/My Drive/20년 2학기/딥러닝/Assignment1/model_checkpoints/problem2_2014-12185.pt')
#print('layer_model saved')

epoch: 1 [1000 / 3125]	 train_loss: 0.728	 train_accuracy: 76.6

epoch: 1 [2000 / 3125]	 train_loss: 0.565	 train_accuracy: 82.8

epoch: 1 [3000 / 3125]	 train_loss: 0.390	 train_accuracy: 89.1

-------- validation --------
accuracy: 87.0

epoch: 2 [1000 / 3125]	 train_loss: 0.566	 train_accuracy: 82.8

epoch: 2 [2000 / 3125]	 train_loss: 0.213	 train_accuracy: 92.2

epoch: 2 [3000 / 3125]	 train_loss: 0.389	 train_accuracy: 85.9

-------- validation --------
accuracy: 87.8

epoch: 3 [1000 / 3125]	 train_loss: 0.331	 train_accuracy: 89.1

epoch: 3 [2000 / 3125]	 train_loss: 0.250	 train_accuracy: 93.8

epoch: 3 [3000 / 3125]	 train_loss: 0.266	 train_accuracy: 90.6

-------- validation --------
accuracy: 89.1

epoch: 4 [1000 / 3125]	 train_loss: 0.226	 train_accuracy: 93.8

epoch: 4 [2000 / 3125]	 train_loss: 0.280	 train_accuracy: 92.2

epoch: 4 [3000 / 3125]	 train_loss: 0.309	 train_accuracy: 92.2

-------- validation --------
accuracy: 89.1

epoch: 5 [1000 / 3125]	 train_loss: 0.32